### 1. 경사 하강법을 이용해서 행렬 분해

In [1]:
import numpy as np

In [2]:
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]
])
R.shape

(4, 5)

In [3]:
num_users, num_items = R.shape
k = 3

In [4]:
## P, Q 의 매트릭스 크기를 지정하고 만들어보기
np.random.seed(11)

P = np.random.normal(scale=1./k, size=(num_users, k))
Q = np.random.normal(scale=1./k, size=(num_items, k))

In [5]:
P.shape, Q.shape

((4, 3), (5, 3))

In [6]:
from sklearn.metrics import mean_squared_error

In [7]:
def get_rmse(R, P, Q, non_zeros):
    error = 0

    # 예측 R을 생성
    full_pred_matrix = np.dot(P, Q.T) # 4x5행렬

    # 오차구하기 - 예측 R과 실제 R의 RMSE 계산해보기 - 실제에서 0인 데이터는 제외
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [8]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)


### iteration step :  0  rmse :  3.2296439041815526
### iteration step :  50  rmse :  1.1167037955327597
### iteration step :  100  rmse :  0.11912292863572094
### iteration step :  150  rmse :  0.04138989896525064
### iteration step :  200  rmse :  0.022844307909711682
### iteration step :  250  rmse :  0.017330041906230807
### iteration step :  300  rmse :  0.015709890520182127
### iteration step :  350  rmse :  0.015191799101326026
### iteration step :  400  rmse :  0.01500076453534186
### iteration step :  450  rmse :  0.014919702708284549
### iteration step :  500  rmse :  0.014879578313867103
### iteration step :  550  rmse :  0.014855560491447746
### iteration step :  600  rmse :  0.014838008002827315
### iteration step :  650  rmse :  0.014823031492216865
### iteration step :  700  rmse :  0.014809037858809965
### iteration step :  750  rmse :  0.014795374897778301
### iteration step :  800  rmse :  0.014781780730006088
### iteration step :  850  rmse :  0.014768156191675869
##

In [9]:
pred_matrix = np.dot(P, Q.T)
print("최종 예측 평점 행렬", np.round(pred_matrix, 2))

최종 예측 평점 행렬 [[ 3.99  1.23  0.64  1.99 -0.94]
 [ 1.11  4.97  2.6   3.    1.01]
 [-3.09  5.47  2.99  3.98  3.98]
 [ 4.98  2.    0.99  1.99 -1.56]]
